In [3]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [4]:
import tensorflow as tf
import os
from keras.utils import np_utils
import numpy as np
import cv2
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPooling2D, Dropout, Flatten, BatchNormalization

from keras.optimizers import Adam
from keras.callbacks import Callback

from keras.utils import np_utils
from keras.datasets import cifar10

import numpy as np
import matplotlib.pyplot as plt

from google.colab import files

from keras import backend as K
K.set_image_data_format('channels_last')

import glob
import matplotlib.pyplot as plt
import sklearn
import random
from PIL import Image
!pip install pydicom
import pydicom
from sklearn.model_selection import train_test_split
from keras.optimizers import Adam
from nibabel.testing import data_path
import nibabel as nib
from google.colab.patches import cv2_imshow

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 47.1 MB/s eta 0:00:00


In [5]:
Data = ['/content/drive/MyDrive/Colab Notebooks/COVID_Project/Subject (101)/*',
        '/content/drive/MyDrive/Colab Notebooks/COVID_Project/Subject (102)/*',
        '/content/drive/MyDrive/Colab Notebooks/COVID_Project/Subject (103)/*',
        '/content/drive/MyDrive/Colab Notebooks/COVID_Project/Subject (104)/*',
        '/content/drive/MyDrive/Colab Notebooks/COVID_Project/Subject (105)/*',
        '/content/drive/MyDrive/Colab Notebooks/COVID_Project/Subject (106)/*',
        '/content/drive/MyDrive/Colab Notebooks/COVID_Project/Subject (107)/*',
        '/content/drive/MyDrive/Colab Notebooks/COVID_Project/Subject (108)/*',
        '/content/drive/MyDrive/Colab Notebooks/COVID_Project/Subject (1000)/*',
        '/content/drive/MyDrive/Colab Notebooks/COVID_Project/Subject (1001)/*',
        '/content/drive/MyDrive/Colab Notebooks/COVID_Project/Subject (1002)/*',
        '/content/drive/MyDrive/Colab Notebooks/COVID_Project/Subject (1003)/*',
        '/content/drive/MyDrive/Colab Notebooks/COVID_Project/Subject (1004)/*',
        '/content/drive/MyDrive/Colab Notebooks/COVID_Project/Subject (1006)/*',
        '/content/drive/MyDrive/Colab Notebooks/COVID_Project/Subject (1007)/*',
        '/content/drive/MyDrive/Colab Notebooks/COVID_Project/Subject (1008)/*',
        '/content/drive/MyDrive/Colab Notebooks/COVID_Project/Subject (1009)/*',
        '/content/drive/MyDrive/Colab Notebooks/COVID_Project/Subject (1010)/*',
        '/content/drive/MyDrive/Colab Notebooks/COVID_Project/Subject (1059)/*',
        '/content/drive/MyDrive/Colab Notebooks/COVID_Project/Subject (1060)/*',
        '/content/drive/MyDrive/Colab Notebooks/COVID_Project/Subject (1061)/*',
        '/content/drive/MyDrive/Colab Notebooks/COVID_Project/Subject (1062)/*',
        '/content/drive/MyDrive/Colab Notebooks/COVID_Project/Subject (1063)/*',
        '/content/drive/MyDrive/Colab Notebooks/COVID_Project/Subject (1064)/*',
        '/content/drive/MyDrive/Colab Notebooks/COVID_Project/Subject (1065)/*',]


data = []
for i in range(len(Data)):
  for name in glob.glob(Data[i]):
    image  = pydicom.dcmread(name)
    image = image.pixel_array.astype(float)
    new_image = cv2.resize(image, (128, 128))
    data.append(new_image)

In [6]:
positive_cases = data
len(positive_cases)

positive_cases = np.array(positive_cases)/255

print(positive_cases.shape)
print(positive_cases.ndim)

(8537, 128, 128)
3


In [7]:
normal_path = '/content/drive/MyDrive/Colab Notebooks/COVID_Project/Normal/*.jpg'
normal_data = []
for name in glob.glob(normal_path):
  image  = cv2.imread(name, cv2.IMREAD_GRAYSCALE)

  new_image = cv2.resize(image, (128, 128))
  normal_data.append(new_image)

len(normal_data)

3840

In [8]:
normal_data = np.array(normal_data)/255
print(normal_data.shape)
print(normal_data.ndim)



(3840, 128, 128)
3


In [9]:
Data  = [normal_data, positive_cases]

normal_labels = np.zeros(len(normal_data)) # labell '0' for normal cases
positive_labels = np.ones(len(positive_cases)) # label '1' for positive cases

#now the data contains 8537 positive cases and 3840 normal cases along with their labels




In [10]:
#Lets shuffle the data and labels:
new_labels = np.concatenate((normal_labels, positive_labels))
from random import shuffle
shuffle(new_labels)

new_data = np.concatenate((Data[0], Data[1]))




#Final Data and Labels:
Y = new_labels
X = new_data.reshape(12377, 128, 128, 1)
shuffle(X)

print(X.shape)
print(Y.shape)

(12377, 128, 128, 1)
(12377,)


In [11]:
Y[0:13]

array([0., 1., 0., 1., 0., 0., 1., 1., 1., 1., 1., 1., 1.])

In [12]:
nRows,nCols, nDims = X.shape[1:]
input_shape = (nRows, nCols, nDims)
print(input_shape)

Y_new = np_utils.to_categorical(Y)
print(Y_new.shape)

(128, 128, 1)
(12377, 2)


In [13]:
Y_new[0:13]

array([[1., 0.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.]], dtype=float32)

In [14]:
#Splitting the dataset into training and test sets:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y_new, test_size = 0.20, random_state=42)

print(len(new_data))
print(len(X_train))
print(Y_train.shape)
print(len(X_test))
print(Y_test.shape)
print(X_test.shape)

12377
9901
(9901, 2)
2476
(2476, 2)
(2476, 128, 128, 1)


In [15]:

#@title Building architecture for Image Classification:

#Input layer in Architecture:

def Model():
  model = Sequential()
  model.add(Conv2D(16, (7, 7), padding = 'same', activation = 'relu',kernel_initializer = 'he_normal', input_shape = input_shape))
  model.add(Conv2D(16, (7,7), activation = 'relu',kernel_initializer = 'he_normal', padding = 'same'))
  model.add(BatchNormalization())
  model.add(MaxPooling2D(pool_size = (2, 2)))
  

  model.add(Conv2D(32, (7, 7), padding = 'same', activation = 'relu', kernel_initializer='he_normal'))
  model.add(Conv2D(32, (7,7), activation = 'relu', padding = 'same', kernel_initializer='he_normal'))
  model.add(BatchNormalization())
  model.add(MaxPooling2D(pool_size = (2, 2)))
  

  model.add(Conv2D(64, (3, 3), padding = 'same', activation = 'relu', kernel_initializer='he_normal'))
  model.add(Conv2D(64, (3, 3), activation = 'relu', padding = 'same', kernel_initializer='he_normal'))
  model.add(BatchNormalization())
  model.add(MaxPooling2D(pool_size = (2, 2)))
  

  model.add(Conv2D(128, (9, 9), padding = 'same', activation = 'relu', kernel_initializer='he_normal'))
  model.add(Conv2D(128, (9, 9), activation = 'relu', padding = 'same', kernel_initializer='he_normal'))
  model.add(BatchNormalization())
  model.add(MaxPooling2D(pool_size = (2, 2)))
  

  model.add(Conv2D(256, (1, 1), padding = 'same', activation = 'relu', kernel_initializer='he_normal'))
  model.add(Conv2D(256, (1, 1), activation = 'relu', padding = 'same', kernel_initializer='he_normal'))
  model.add(BatchNormalization())
  model.add(MaxPooling2D(pool_size = (2, 2)))
  

  model.add(Conv2D(512, (3, 3), padding = 'same', activation = 'relu', kernel_initializer='he_normal'))
  model.add(Conv2D(512, (3, 3), activation = 'relu', padding = 'same', kernel_initializer='he_normal'))
  model.add(BatchNormalization())
  

  model.add(Flatten())
  model.add(Dense(512, activation = 'relu'))
  model.add(Dense(2, activation = 'sigmoid'))

  return model  

In [16]:
K.clear_session()
model = Model()
batch = 256
epochs = 25

In [17]:
adam = Adam(learning_rate  = 0.0000001)
model.compile(loss = 'binary_crossentropy', optimizer = adam, metrics = ['accuracy'])

In [ ]:
# #Model fitting:
# #Modelcheckpoint:
# Checkpointer = tf.keras.callbacks.ModelCheckpoint('model_for_HRCT.h5', verbose = 1, save_best_only=True)
# reduce  = tf.keras.callbacks.ReduceLROnPlateau(monitor = 'val_loss', factor = 0.2, patience = 2, min_lr = 0.000000001, verbose = 1)

# logdir = 'HRCT'

# tensorboard_Vis = tf.keras.callbacks.TensorBoard(log_dir = logdir)

history = model.fit(X_train, Y_train, validation_split = 0.25, batch_size = 256, epochs = 20)#, callbacks = [Checkpointer, reduce, tensorboard_Vis])

Epoch 1/20
 7/30 [======>.......................] - ETA: 25:41 - loss: 0.9066 - accuracy: 0.4308

In [ ]:
plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label = 'val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0.5, 1])
plt.legend(loc='lower right')

In [ ]:
model.evaluate(X_test, Y_test, verbose = 1)

In [ ]:
pred = model.predict(X_test)
y_classes = [np.argmax(element) for element in pred]
print('Predicted labels:',y_classes[0:20])
print('Actual labels:',Y_test[0:20])

In [ ]:
from tensorflow import keras
from keras.utils.vis_utils import plot_model

keras.utils.plot_model(model, to_file = 'model_HRCT.png', show_layer_names = True)

In [ ]:
test_loss, test_acc = model.evaluate(X_test,  Y_test, verbose=2)

In [ ]:
logdir = 'HRCT'

%load_ext tensorboard
%tensorboard --logdir HRCT/

In [ ]:
A = model.predict(X_test)
predictions = np.argmax(A, axis = 1)
print("Actual Label of Image", Y_test[10:20])
print("Predicted Label of Image", predictions[10:20])